# chapter 6. 분류2 KNN
1. **KNN 원리**: 거리 기반 다수결 분류
2. **K 값의 영향**: K가 작으면 과적합, K가 크면 과소적합
3. **스케일링의 중요성**: 단위가 다르면 거리 계산이 왜곡됨
4. **Decision Boundary 시각화**: K에 따라 경계가 어떻게 변하는지
5. **Pipeline + GridSearchCV**: 최적의 K 찾기

# 오늘 배운 내용 먼저 정리
1. KNN은 가까운 이웃 K개를 찾아 투표로 분류하는 거리 기반 모델이다.
2. K가 작으면 경계가 복잡해져 과적합 쪽, K가 크면 경계가 단순해져 과소적합 쪽으로 간다.
3. 거리 기반이므로 스케일링이 매우 중요하고, 보통 Pipeline으로 묶어 누수를 막는다.
4. 정확도만 보지 말고 혼동행렬로 어떤 클래스를 헷갈리는지 확인한다.
5. K 선택은 단발성 테스트보다 교차검증과 GridSearchCV로 평균 성능과 흔들림까지 보고 결정한다.
6. 고차원에서는 거리의 구분력이 약해져 KNN이 약해질 수 있다.(차원의 저주)

## part 1. KNN이란?
KNN은 분류에서 가장 직관적인 기준을 가진 모델이다.

새 데이터 x가 들어오면, 훈련 데이터에서 x와 가까운 K개의 이웃을 찾고,\
그 이웃들의 라벨을 보고 다수결(또는 가중치 투표)로 클래스를 정한다.
> 모델을 먼저 학습해서 규칙을 만드는 게 아닌, 필요할 때마다 가까운 데이터 검색 → 투표로 예측하는 방식


### K(이웃 수)의 의미 (중요)
- K가 작다: 아주 가까운 몇 개만 보고 판단한다. → 훈련 데이터에 민감해지기 쉽다. (과적합 위험 ↑)
- K가 크다: 더 많은 이웃을 평균내어 본다. → 경계가 부드러워지지만 뭉개질 수 있다. (과소적합 위험 ↑)
- K(이웃 수)는 "얼마나 주변을 넓게 보겠다"를 표현하는 정책값이다.
> 팁: 동률 방지를 위해 K를 홀수로 시작하는 경우가 많고, 최종 K는 교차검증으로 고른다. (K는 하이퍼파라미터)

### 가까운 이웃을 구하는 방식: 유클리드 거리
가까운 이웃을 구하는 방식은 가장 기본은 유클리드 거리다. (직선 거리)\
sklearn에선 보통 `metric="minkowski", p=2`가 유클리드 거리

#### 다른 방법들은 참고
1) 맨해튼 거리(Manhattan): 직선이 아니라 가로+세로로만 이동한다고 생각하면 된다. (도시 블록에서 길 따라 걷는 느낌)
2) 코사인 유사도(Cosine): 거리 보단 방향이 비슷한지 본다. 특히, 텍스트나 임베딩처럼 **벡터 방향**이 중요할 때 자주 사용한다.
3) 해밍 거리(Hamming): 0/1 같은 데이터에서 서로 다른 칸이 몇 개인지 세는 방식이다. (예: 10101 vs 11100 → 다른 자리 개수 세기)

### 왜 표준화(스케일링)가 중요하냐? (진짜 중요)
거리 계산은 숫자 크기에 민감하다.\
예를 들어 키: 170 ~ 190 (차이 20)와 연봉: 3,000만 ~ 8,000만 (차이 5,000만)을 같이 넣으면 연봉의 값이 압도적으로 크기에 거리를 거의 지배해버린다.\
그래서 KNN은 보통 스케일링이 필수에 가깝다.

### KNN 예시 k=3
새 데이터 *의 이웃 k(3)개:
빨강 (거리 = 1.2)\
빨강 (거리 = 1.5)\
파랑 (거리 = 2.0)

다수결: 빨강 2 > 파랑 1 → * = 삘강

### KNN이 분류하는 흐름
1. 거리 정의
    - 보통 연속형 특성에서는 유클리드 거리 사용
    - 다른 거리도 가능(맨해튼, 코사인 등)
2. 가까운 K개 이웃 찾기
    새 샘플 x에 대해 훈련 데이터 전체와의 거리를 계산하고 가까운 K개 선택
3. 다수결(또는 거리 가중치)로 예측
    - 기본: K개 중 가장 많은 라벨
    - 변형: 더 가까운 이웃에 가중치를 더 준다(distance-weighted)

> KNN은 학습보단, 필요할 때마다 "검색해 투표"하는 방법이다.

## part 2. Iris로 KNN 기본 사용법
아래 코드는 Iris 데이터를 train/test로 나누고, KNN으로 분류 성능을 확인한다.

사용 함수/메소드 요약:
- `load_iris()`: 예제 데이터 로드
- `train_test_split(..., stratify=y)`: 클래스 비율 유지하며 분리
- `KNeighborsClassifier(...)`: KNN 모델 생성
- `fit`, `predict`: 학습/예측
- `accuracy_score`, `classification_report`: 성능 요약

In [2]:
import numpy as np, pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# 예제 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target # 입력값, 정답값 설정

X_train, X_test, y_train, y_test = train_test_split( # train/test로 값 분리
    X, 
    y, 
    test_size=0.2, 
    stratify=y,         #  클래스 비율 유지하며 분리
    random_state=42
)

model = KNeighborsClassifier( # KNN 모델 생성
    n_neighbors=5, # 투표에 참여할 이웃 수(K값)
    metric="minkowski", # 유클리드 거리로 설정
    p=2  # p=2 → 거리값 설정
)

model.fit(X_train, y_train) # 학습 실행

y_pred = model.predict(X_test) # 예측 실행

print(f"[Test Accuracy] {accuracy_score(y_test, y_pred):.3f}") # 성능 요약
print()
print(classification_report( # 클래스별 precision/recall/f1과 전체 평균
    y_test, 
    y_pred, 
    target_names=iris.target_names, 
    digits=3))

[Test Accuracy] 1.000

              precision    recall  f1-score   support

      setosa      1.000     1.000     1.000        10
  versicolor      1.000     1.000     1.000        10
   virginica      1.000     1.000     1.000        10

    accuracy                          1.000        30
   macro avg      1.000     1.000     1.000        30
weighted avg      1.000     1.000     1.000        30



## 혼동행렬(Confusion Matrix)로 어디서 틀렸는지 보기
정확도만 보지 말고, 어떤 클래스를 어떤 클래스로 틀리는지 확인한다.

- confusion_matrix: 실제/예측 교차표
- 히트맵으로 시각화

In [3]:
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

cm = confusion_matrix(y_test, y_pred)

fig = ff.create_annotated_heatmap(
    z=cm,
    x=list(iris.target_names),
    y=list(iris.target_names),
    colorscale="Blues", showscale=True
)
fig.update_layout(
    title="Confusion Matrix (KNN, K=5)",
    xaxis=dict(title="Predicted"),
    yaxis=dict(title="Actual"),
    template="plotly_dark"
)
fig.show()

### Confusion Matrix 해석하기 (KNN, K=5, Iris 데이터)
혼동행렬은 모델이 어디서 맞고, 어디서 틀렸는지 보여주는 표다.

- 행(row) = 실제값(Actual)
- 열(column) = 예측값(Predicted)

- 대각선 = 맞춘 개수
- 대각선이 아닌 곳 = 틀린 개수

지금 결과는 대각선만 값이 있고, 나머지는 0이다.\
→ 모든 데이터를 정확히 분류했다는 의미다.

### KNN 관점에서 해석
KNN은 가까운 이웃 K개의 다수결로 결정한다. 이 때문에 경계 근처 데이터는 섞일 수 있다.\
(K 값이 너무 작으면 과적합, K 값이 너무 크면 과소적합)\
지금의 결과는 K=5가 적절했고 데이터 분리가 잘 되었다 볼수 있다.


## part 3. 재차 강조하는 KNN에서 스케일링이 왜 중요한가
KNN은 거리로 판단한다. 때문에 피처 단위가 다르면 거리가 왜곡되어 성능이 흔들린다.

스케일(단위)이 다르면 **큰 단위**가 거리 계산을 지배한다.\
즉, 의도와 다르게 한 변수만 보고 분류하는 모델이 되어버릴 수 있다.

스케일링을 하면 **모든 변수의 영향력을 비슷하게 맞춘다**\
각 변수를 비슷한 크기 범위로 맞춰서, 거리 계산이 특정 변수에만 끌려가지 않게 한다.\
때문에 KNN에서 스케일링은 필수 작업이다.


일부로 두 피처의 스케일을 크게 다르게 만들어 스케일링 전/후 성능 차이를 비교해보자.

In [4]:
from sklearn.preprocessing import StandardScaler

rng_scale = np.random.RandomState(42)
X_demo = np.c_[rng_scale.rand(200), rng_scale.rand(200) * 1000]
y_demo = (X_demo[:, 0] + X_demo[:, 1] / 1000 > 1.0).astype(int)

Xd_tr, Xd_te, yd_tr, yd_te = train_test_split(
    X_demo, 
    y_demo, 
    test_size=0.3, 
    stratify=y_demo, 
    random_state=42
)

acc_raw = accuracy_score(
    yd_te,
    KNeighborsClassifier(5).fit(Xd_tr, yd_tr).predict(Xd_te)
)

sc = StandardScaler()               # 평균 0, 표준편차 1로 표준화
Xd_tr_s = sc.fit_transform(Xd_tr)   # train 기준으로 스케일러를 학습 (train 에만)
Xd_te_s = sc.transform(Xd_te)       # train 기준으로 스케일러를 적용 (test에는 transform만) (데이터 누수 방지)
acc_scaled = accuracy_score(
    yd_te,
    KNeighborsClassifier(5).fit(Xd_tr_s, yd_tr).predict(Xd_te_s)
)

pd.DataFrame({
    "설정": ["스케일링 없음", "스케일링(StandardScaler)"],
    "Accuracy": [acc_raw, acc_scaled]
})

,설정,Accuracy
0,스케일링 없음,0.650000
1,스케일링(StandardScaler),0.983333


스케일링 전후 Accuracy 차이가 극적으로 발생한다.\
즉, KNN에서 스케일링을 하지않으면 성능이 하락한다는 것을 확인할 수 있다.

### 안전하게 스케일링하는 법: Pipeline

In [5]:
from sklearn.pipeline import Pipeline

pipe_knn = Pipeline([ # 전처리 → 모델 순서로 실행
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=5))
])

knn_raw = KNeighborsClassifier(n_neighbors=5)
acc_raw = accuracy_score(y_test, knn_raw.fit(X_train, y_train).predict(X_test))

acc_pipe = accuracy_score(y_test, pipe_knn.fit(X_train,y_train).predict(X_test))

pd.DataFrame({
    "Model": ["KNN (스케일링 없음)", "Pipeline (Scaler + KNN)"],
    "Accuracy": [acc_raw, acc_pipe]
})

,Model,Accuracy
0,KNN (스케일링 없음),1.000000
1,Pipeline (Scaler + KNN),0.933333


# part 4. Decision Boundary(결정경계)로 과적합/과소적합 감 잡기
분류모델은 입력값 X를 보고 **A냐 B냐(0이냐 1이냐)**를 결정한다\
이때 A로 판단하는 영역과 B로 판단하는 영역이 생기는데,\
그 **경계선(또는 경계면)**을 디시전 바운더리라 부른다.

> 클래스가 바뀌는 경계

### 2차원으로 접근해 생각해보기
특성이 2개라고 하자. (예: 공부시간, 모의고사 점수)

- 점 하나 = 사람 한 명(데이터 한 개)
- 색 = 라벨(합격/불합격)

이때 모델을 분류하면 평면은 이렇게 나눠진다.\
- 왼쪽 영역: 모델이 불합격(0)
- 오른쪽 영역: 모델이 합격(1)

이때 0 영역과 1 영역이 갈리는 선이 바로 결정경계, 디시전 바운더리다.

### 모델별로 바뀌는 결정경계
- 로지스틱 회귀(기본형)\
결정경계가 보통 직선(또는 평평한 면) 느낌\
그래서 데이터 경계가 휘어져 있으면 한 번에 잘 못 자를 수 있음

- KNN\
이웃을 따라가니까 결정경계가 울퉁불퉁해질 수 있음\
K가 작으면 더 울퉁불퉁(복잡), K가 크면 더 매끈(단순)


### 그래서 디시전 바운더리가 왜 중요한가?: 과적합과 과소적합
- 경계가 너무 구불구불하고 점들을 피해 다니면 → 과적합 가능성 ↑
    - 모델이 너무 단순해서 패턴을 제대로 못 잡는 상태. 결정경계가 너무 단순하다. (대충 직선 한 줄로 끝)
    - 훈련 데이터도 잘 못 맞추고, 테스트 데이터도 못 맞춘다.
        - train/test 점수가 낮음

- 경계가 너무 단순해서 점들을 많이 잘라버리면 → 과소적합 가능성 ↑
    - 모델이 너무 복잡해서 훈련 데이터의 "노이즈"까지 외워버린 상태. 결정경계가 구불구불, 점 하나하나 피해 다니는 느낌
    - 훈련 데이터는 엄청 잘 맞추나 새 데이터(테스트)엔 잘 안 맞음
        - train 점수 높으나 test 점수 낮음 (차이가 큼)

## Part 5. GridSearchCV로 최적의 K 찾기
K를 하나 찍어서 고르는 대신,\
교차검증 기반으로 여러 후보를 공정하게 비교해 최적 조합을 고른다.


### 사용 코드

사용 함수/메서드 요약:
- `GridSearchCV(estimator, param_grid, cv, scoring)`: 후보 조합을 전부 평가
- `best_params_`, `best_score_`: 최적 조합과 그 성능

In [7]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

param_grid = {
    "knn__n_neighbors": [1, 3, 5, 7, 9, 11],
    "knn__weights": ["uniform", "distance"],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="accuracy",
    cv=cv,
    n_jobs=-1,
    refit=True,
    return_train_score=True
)

iris = load_iris()
gs.fit(iris.data, iris.target)

print(f"최적 K: {gs.best_params_}")
print(f"최적 CV Accuracy: {gs.best_score_:.4f}")

최적 K: {'knn__n_neighbors': 5, 'knn__weights': 'uniform'}
최적 CV Accuracy: 0.9733


## part 6. 차원의 저주: KNN은 왜 고차원에서 약해질까
차원이 늘수록(특성 수가 많을수록) "가까움"과 "멀음"의 차이가 희미해진다.\
즉, 가장 가까운 점도 별로 안 가깝게 느껴져서 KNN이 힘들어진다.



### 개념 설명 (KNN 관점)
KNN은 모든 샘플과의 거리를 재고 가장 가까운 K개를 골라 다수결로 분류한다.\
만약 데이터가 고차원이 될 경우...

### 간단 예시(감각만)
특성 2개(2차원)면 "비슷한 점"이 주변에 모여있을 가능성이 크다.\
특성 50개(50차원)면 "모든 특성이 동시에 비슷한 점"을 찾기가 어렵다.

즉, "키도 비슷하고, 몸무게도 비슷하고, 나이도 비슷하고, … 50개가 다 비슷한 사람"을 찾는 상황이 되어버린다.\
현실적으로도, 데이터 적으로도 그런 이웃을 찾기 어렵다.

#### 거리들이 다 비슷해진다.: "가장 가까운 이웃"이 사실상 그렇게 가깝지 않게 된다.
차원이 많아지면, 한 변수에서 조금 가까워도 다른 변수들에서 조금씩 멀어질 확률이 커진다.\
이 "조금씩"이 여러 차원에 쌓여, 결과적으로 
- 제일 가까운 점도
- 그다음 가까운 점도
- 꽤 멀어 보이는 점도

거리 차이가 크게 안 나게 된다.

#### 데이터가 희박(sparse)해진다. : 다수결이 의미가 약해진다.
차원이 늘어날수록 공간의 "부피"가 엄청 커진다.\
그런데 데이터 개수는 보통 그대로니까, 점들이 공간에 드문드문 흩어져 있게 된다.
그래서\
내 주변에 진짜 가까운 이웃이 거의 없고\
이웃을 찾으려면 반경을 크게 키워야 하는데\
그러면 이웃이 "비슷한 애들"이 아닌 "그냥 아무나"가 된다.